## 음식 카테고리 별 평균 리뷰 수 구하기
- pair RDD 생성, reduceByKey(), mapValues()

In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf().setMaster("local").setAppName("category-review-average")
sc = SparkContext(conf=conf)

22/03/11 23:21:03 WARN Utils: Your hostname, sinjiyeong-ui-iMac.local resolves to a loopback address: 127.0.0.1; using 220.79.230.51 instead (on interface en0)
22/03/11 23:21:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/11 23:21:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
directory = "/Users/shinjiyoung/PycharmProjects/data-engineering/01-spark/data"
filename = "restaurant_reviews.csv"

In [15]:
lines = sc.textFile(f"file:///{directory}/{filename}")

In [16]:
lines.collect()

['id,item,cateogry,reviews',
 '0,짜장면,중식,125',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223   ',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [21]:
header = lines.first()
header

'id,item,cateogry,reviews'

In [19]:
filtered_lines = lines.filter(lambda row: row != header)

In [20]:
filtered_lines.collect()

['0,짜장면,중식,125',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223   ',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [22]:
def parse(row):
    # 0,짜장면,중식,125
    fields = row.split(",")
    category = fields[2]
    reviews = int(fields[3])
    return (category, reviews)

In [23]:
category_reviews = filtered_lines.map(parse)

In [24]:
category_reviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [25]:
category_reviews_count = category_reviews.mapValues(lambda x: (x, 1))

In [26]:
category_reviews_count.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [30]:
reduced = category_reviews_count.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

In [31]:
reduced.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [33]:
averages = reduced.mapValues(lambda x: x[0] / x[1])

In [35]:
averages.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]